In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import matplotlib.pyplot as plt

# Load your dataset and preprocess it as needed
# X, y = load_data()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define a baseline neural network model
def baseline_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Train and evaluate the baseline model
baseline_model = baseline_model()
baseline_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_split=0.2)
baseline_scores = baseline_model.evaluate(X_test, y_test, verbose=0)
print(f"Baseline Accuracy: {baseline_scores[1]*100:.2f}%")

# Define a function to create a neural network model with hyperparameters
def create_model(learning_rate=0.001, dropout_rate=0.2, weight_decay=1e-4):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=keras.regularizers.l1_l2(weight_decay)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(weight_decay)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Hyperparameter tuning using RandomizedSearchCV
param_dist = {
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout_rate': [0.2, 0.3, 0.4],
    'weight_decay': [1e-4, 1e-5, 1e-6],
    'batch_size': [16, 32, 64],
    'epochs': [20, 30, 40]
}

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = KerasClassifier(build_fn=create_model, verbose=0)
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=kfold, scoring='accuracy', verbose=1, n_jobs=-1)
random_search_results = random_search.fit(X_train, y_train)

print(f"Best Accuracy: {random_search_results.best_score_:.2f}%")
print(f"Best Parameters: {random_search_results.best_params_}")

# Neural network with optimal hyperparameters
best_model = random_search_results.best_estimator_
best_model.fit(X_train, y_train, epochs=best_model.epochs, batch_size=best_model.batch_size, verbose=1, validation_split=0.2)
best_scores = best_model.model.evaluate(X_test, y_test, verbose=0)
print(f"Best Model Accuracy: {best_scores[1]*100:.2f}%")

# Cross-validation with the best model
cv_scores = []
for train, val in kfold.split(X_train, y_train):
    cv_model = create_model(learning_rate=best_model.learning_rate, dropout_rate=best_model.dropout_rate, weight_decay=best_model.weight_decay)
    cv_model.fit(X_train[train], y_train[train], epochs=best_model.epochs, batch_size=best_model.batch_size, verbose=0)
    _, accuracy = cv_model.evaluate(X_train[val], y_train[val], verbose=0)
    cv_scores.append(accuracy)

print(f"Cross-Validation Accuracy: {np.mean(cv_scores)*100:.2f}%")

# Ensemble methods using Bagging
from sklearn.ensemble import BaggingClassifier

bagging_model = BaggingClassifier(base_estimator=best_model, n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)
bagging_accuracy = bagging_model.score(X_test, y_test)
print(f"Bagging Accuracy: {bagging_accuracy*100:.2f}%")

# Learning rate scheduling
def learning_rate_schedule(epoch, learning_rate):
    if epoch < 10:
        return learning_rate
    else:
        return learning_rate * tf.math.exp(-0.1)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(learning_rate_schedule)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

model = create_model()
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[lr_schedule, early_stopping, model_checkpoint])

# Plot learning curve
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
